In [1]:
from datasets import Dataset
from transformers import BertTokenizerFast, BertForQuestionAnswering,Trainer, TrainingArguments
import torch
import json


d:\Projects\LLM\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("../datasets/ChatGPT/extractive/fridge_dataset_v1.0_clean.json", "r") as f:
    data = json.load(f)

In [3]:
# Перетворення під BERT-формат
rows = []
for item in data:
    context = item["context"]
    question = item["question"]
    if item["answers"]:
        answer_start = item["answers"][0]["answer_start"]
        answer = item["answers"][0]["text"]
        rows.append({
            "context": context,
            "question": question,
            "answers": {"text": [answer], "answer_start": [answer_start]},
            "is_impossible": item["is_impossible"]
        })
    else:
        # Якщо відповіді немає, можна пропустити запис або додати порожні значення
        rows.append({
            "context": context,
            "question": question,
            "answers": {"text": [""], "answer_start": [0]},
            "is_impossible": item["is_impossible"]
        })

# Створення Dataset
dataset = Dataset.from_list(rows)

# Перевірка
print(dataset[-1])

{'context': 'Maintenance\nCleaning\nInterior and exterior\nвљ\xa0 WARNING\nвЂў Do not use benzene, thinner, vinegar, liquid bleach, baking soda, citric, salt mixtures or home/car detergent such as Cloroxв„ў for cleaning purposes. They may damage the surface of the refrigerator and cause a fire.\nвЂў Do not spray water onto the refrigerator. This may cause electric shock.\nRegularly use a dry cloth to remove all foreign substances such as dust or water from the power plug terminals and contact points.\n1. Unplug the power cord.\n2. Use a moistened, soft, lint-free cloth or paper towel to clean the refrigeratorвЂ™s interior and exterior.\n3. When done, use a dry cloth or paper towel to dry well.\n4. Plug in the power cord.\n\nLED Lamps\nTo replace the lamps of the refrigerator, contact a local Samsung service centre.\nвљ\xa0 WARNING\nThe lamps are not user-serviceable. Do not attempt to replace a lamp yourself. This can cause electric shock.\n\nLamp (Light Source)\nThis product contains 

In [4]:
# 3. Тренувальний/валідаційний спліт
split_dataset = dataset.train_test_split(test_size=0.15, seed=42)

In [5]:
# 4. Токенізатор і модель
tokenizer = BertTokenizerFast.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

d:\Projects\LLM\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nobla\.cache\huggingface\hub\models--bert-large-uncased-whole-word-masking-finetuned-squad. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back t

In [23]:
def preprocess(examples):
    # Tokenize questions and contexts
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=512,
        truncation="only_second",
        padding="max_length",
        return_tensors=None  # Remove return_tensors="pt" for batched processing
    )
    
    # Get start positions and answer texts
    start_positions = []
    end_positions = []
    
    # Process each example in the batch
    for i in range(len(examples["question"])):
        answer_start = examples["answers"][i]["answer_start"][0]
        answer_text = examples["answers"][i]["text"][0]
        
        # Get offsets for this example
        tokenized_context = tokenizer(
            examples["context"][i],
            return_offsets_mapping=True,
            max_length=512,
            truncation=True,
            padding="max_length",
        )
        
        offsets = tokenized_context["offset_mapping"]
        
        # Find start/end token indices
        start_idx = None
        end_idx = None
        
        for idx, (start, end) in enumerate(offsets):
            if start <= answer_start < end:
                start_idx = idx
                break
                
        if start_idx is not None:
            for idx, (start, end) in enumerate(offsets[start_idx:], start=start_idx):
                if end >= answer_start + len(answer_text):
                    end_idx = idx
                    break
                    
        if start_idx is None or end_idx is None:
            start_idx = 0
            end_idx = 0
            
        start_positions.append(start_idx)
        end_positions.append(end_idx)
    
    # Add start and end positions to inputs
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    
    return inputs

# Test with a single example
single_example = split_dataset["train"].select([0])
tokenized_example = single_example.map(preprocess, batched=True)

# Print result
print("Приклад токенізації для першого запису:")
print(tokenized_example[0])

Map: 100%|██████████| 1/1 [00:00<00:00, 76.02 examples/s]

Приклад токенізації для першого запису:
{'context': 'Instructions about the WEEE\nCorrect Disposal of This Product (Waste Electrical & Electronic Equipment)\n(Applicable in countries with separate collection systems)\nThis marking on the product, accessories or literature indicates that the product and its electronic accessories (e.g. charger, headset, USB cable) should not be disposed of with other household waste at the end of their working life.\nTo prevent possible harm to the environment or human health from uncontrolled waste disposal, please separate these items from other types of waste and recycle them responsibly to promote the sustainable reuse of material resources.\nHousehold users should contact either the retailer where they purchased this product, or their local government office, for details of where and how they can take these items for environmentally safe recycling.\nBusiness users should contact their supplier and check the terms and conditions of the purchase cont

In [25]:
def check_tokenization(example, start_idx, end_idx):
    # Токенізація контексту для виведення
    tokenized_context = tokenizer(
        example["context"],
        return_offsets_mapping=True,
        max_length=512,
        truncation=True,
        padding="max_length"
    )

    # Отримуємо токени контексту
    tokens = tokenizer.convert_ids_to_tokens(tokenized_context["input_ids"])

    # Виводимо контекст між start і end індексами
    print("Context Tokens:")
    print(tokens[start_idx:end_idx+1])  # Вивести частину токенів відповіді

# Перевірка для першого запису
check_tokenization(single_example[0], 156, 190)


Context Tokens:
['business', 'users', 'should', 'contact', 'their', 'supplier', 'and', 'check', 'the', 'terms', 'and', 'conditions', 'of', 'the', 'purchase', 'contract', '.', 'this', 'product', 'and', 'its', 'electronic', 'accessories', 'should', 'not', 'be', 'mixed', 'with', 'other', 'commercial', 'waste', '##s', 'for', 'disposal', '.']


In [ ]:

tokenized_train = split_dataset["train"].map(preprocess, batched=True)
tokenized_val = split_dataset["test"].map(preprocess, batched=True)

In [16]:
# Перевірка перших 1 прикладів
for i in range(1):
    print(tokenized_train[i])


{'context': 'Instructions about the WEEE\nCorrect Disposal of This Product (Waste Electrical & Electronic Equipment)\n(Applicable in countries with separate collection systems)\nThis marking on the product, accessories or literature indicates that the product and its electronic accessories (e.g. charger, headset, USB cable) should not be disposed of with other household waste at the end of their working life.\nTo prevent possible harm to the environment or human health from uncontrolled waste disposal, please separate these items from other types of waste and recycle them responsibly to promote the sustainable reuse of material resources.\nHousehold users should contact either the retailer where they purchased this product, or their local government office, for details of where and how they can take these items for environmentally safe recycling.\nBusiness users should contact their supplier and check the terms and conditions of the purchase contract. This product and its electronic ac

In [9]:
training_args = TrainingArguments(
    output_dir="./bert_finetuned",
    eval_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

In [12]:
# 6. Тренер
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)



D:\TEMP\ipykernel_7348\2797870358.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 7. Навчання
trainer.train()

In [49]:
# --- Крок 5: Функція для обчислення метрик ---

metric = evaluate.load("squad")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    examples = split_dataset["test"]
    features = tokenized_test

    preds = postprocess_qa_predictions(examples, features, logits)

    formatted_preds = [{"id": k, "prediction_text": v} for k, v in preds.items()]
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]

    return metric.compute(predictions=formatted_preds, references=references)

In [50]:
# --- Крок 6: Параметри тренування ---
training_args = TrainingArguments(
    output_dir="./bert_finetuned",
    eval_strategy="steps",
    eval_steps=1,
    logging_steps=1,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

In [51]:
# --- Крок 7: Ініціалізація тренера ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\nobla\AppData\Local\Temp\ipykernel_16412\2147146533.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [52]:
# --- Крок 8: Запуск тренування ---
trainer.train()

Step,Training Loss,Validation Loss,Exact Match,F1
1,4.283200,5.391767,40.000000,64.636364
2,6.382000,4.731651,40.000000,64.636364
3,4.493800,4.244206,40.000000,64.636364
4,4.768600,3.792037,10.000000,57.303030
5,4.972200,3.433322,10.000000,58.303030
6,3.161700,3.204514,10.000000,58.508159
7,4.207100,3.043259,10.000000,55.841492
8,3.156600,2.946510,10.000000,55.841492
9,3.225400,2.884795,10.000000,55.841492
10,3.442600,2.839820,10.000000,57.919414


TrainOutput(global_step=14, training_loss=3.766544818878174, metrics={'train_runtime': 57.9011, 'train_samples_per_second': 1.796, 'train_steps_per_second': 0.242, 'total_flos': 10190941802496.0, 'train_loss': 3.766544818878174, 'epoch': 2.0})

In [53]:
print(tokenized_test.column_names)


['id', 'input_ids', 'attention_mask', 'offset_mapping', 'example_id', 'start_positions', 'end_positions']


In [54]:
# --- Крок 7: Збереження моделі ---
trainer.save_model("./bert_finetuned")